In [48]:
using JLD

In [49]:
# tic()

In [50]:
# Определение параметров дискретизации

c = 299792458; #Скорость света [м/с]
mu0 = 4*pi*10.0^-7; # Магнитная проницаемость
eps0 = 1/(mu0*c*c); # Диэлектрическая проницаемость

# c = 3e8; #Скорость света [м/с]
frequency = 2.4e9; # Базовая частота [Гц]
wavelength = c/frequency; # Длина волны в вакууме [м]

# # стекло
# eps_media=4.7
# mu_media=0.999987
# sig_media=1e-11

# медь
eps_media=1.
mu_media=0.999994
sig_media=59.5e6
# sig_media=0.1


n_coef=sqrt(eps_media*mu_media)

wavelength_in_media = c/n_coef/frequency; # Длина волны в вакууме [м]

@printf "f = %.3f MHz - frequency\n" frequency/10^6;
@printf "λ = %.3f m - wavelength\n" wavelength;
@printf "λ = %.3f m - wavelength in media\n" wavelength_in_media;

# dz = wavelength/100 # Шаг дискретизации пространства [м]
dz = wavelength_in_media/50 # Шаг дискретизации пространства [м]
dt = dz/c # Шаг дискретизации времени [с], Магическое соотношение

@printf "%.5f mm - space step\n" dz*1000;
@printf "%.3fE-6 us - time step\n" dt*10^12;

# nlength = Int(round(10*wavelength/dz)) # Количество точек разбиения пространства
# xSteps = 2000 # Количество точек разбиения пространства
# timeSteps = 5000 # Количество временных шагов

S_domain=4
T_domain=0.025e-6

# @printf "%.3f m - space domain length\n" xSteps*dz;
# @printf "%.4f us - time domain length\n" timeSteps*dt*1000000;
@printf "%.3f m - space domain length\n" S_domain;
@printf "%.4f us - time domain length\n" T_domain*1000000;

xSteps = Int(round(S_domain/dz)) # Количество точек разбиения пространства
timeSteps = Int(round(T_domain/dt)) # Количество временных шагов
@printf "%d - number of space steps\n" xSteps;
@printf "%d - number of time steps\n" timeSteps;

skin_length=sqrt(2/sig_media/(mu_media*mu0)/(2*pi*frequency))
@printf "%.15f - skin effect length\n" skin_length;

f = 2400.000 MHz - frequency
λ = 0.125 m - wavelength
λ = 0.125 m - wavelength in media
2.49828 mm - space step
8.333E-6 us - time step
4.000 m - space domain length
0.0250 us - time domain length
1601 - number of space steps
3000 - number of time steps
0.000001331856182 - skin effect length


In [51]:
# Инициализация массивов проводимости, диэлектрической и магнитной проницаемости
sigma  = zeros(xSteps);
epsilon = ones(xSteps);
mu = ones(xSteps);
# Инициализация массивов коэффициентов
Ca = zeros(xSteps);
Cb = zeros(xSteps);
Da = zeros(xSteps);
Db = zeros(xSteps);
# Инициализация массивов полей
Ex=zeros(timeSteps,xSteps)
Hy=zeros(timeSteps,xSteps)
# Пространсвенная ось
# x = linspace(0,wavelength*10,nlength)
x = linspace(dz,xSteps*dz,xSteps)
# Ось времени
t= linspace(dt,timeSteps*dt,timeSteps);

In [52]:
# Начальные условия
Nu, delta = x[end]/4, wavelength/4 #Переменные ню и дельта для начального распределения Гаусса
G=exp(-((x-Nu).^2)/(2*delta^2)) #Начальное распределение Гаусса
# Параметры среды
left_bord_ind=Int(round(length(x)/4*3))
right_bord_ind=Int(round(length(x)/5*4));
# left_bord_ind=1;
# right_bord_ind=length(x);
sigma[left_bord_ind:right_bord_ind]=sig_media;
epsilon[left_bord_ind:right_bord_ind]=eps_media;
# mu[left_bord_ind:right_bord_ind]=12.6;
mu[left_bord_ind:right_bord_ind]=mu_media;

# Ex[1,:]=G;
# Hy[1,:]=1/120pi*G;

In [53]:
function signal_gen(t,f)
#     signal=cos(2*pi*f/50.*(t-dt)).*sin(2*pi*f.*(t-dt))
#     signal=sin(2*pi*f.*(t-dt))
    signal=sin(2*pi*f.*(t-dt)).*exp(-((t-t[Int(round(length(t)/4))]).^2)./(2*(1./frequency.*2).^2))
#     exp(-((x-Nu).^2)/(2*delta^2))
    return signal
end

signal_gen (generic function with 1 method)

In [54]:
# signal=zeros(t)
signal=signal_gen(t,frequency);
signal_pos=Int(round(length(x)/4))

400

In [55]:
# using Plots
# gr()
plot(t*10^6,signal)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,194.219 39.6781,194.219 39.8637,194.219 40.0493,194.219 40.2349,194.219 40.4205,194.219 40.6061,194.219 40.7917,194.219 40.9772,194.219 41.1628,194.219 
 41.3484,194.219 41.534,194.219 41.7196,194.219 41.9052,194.219 42.0908,194.219 42.2763,194.219 42.4619,194.219 42.6475,194.219 42.8331,194.219 43.0187,194.219 
 43.2043,194.219 43.3899,194.219 43.5754,194.219 43.761,194.219 43.9466,194.219 44.1322,194.219 44.3178,194.219 44.5034,194.219 44.689,194.219 44.8745,194.219 
 45.0601,194.219 45.2457,194.219 45.4313,194.219 45.6169,194.219 45.8025,194.219 45.9881,194.219 46.1736,194.219 46.3592,194.219 46.5448,194.219 46.7304,194.219 
 46.916,194.219 47.1016,194.219 47.2871,194.219 47.4727,194.219 47.6583,194.219 47.8439,194.219 48.0295,194.219 48.2151,194.219 48.4007,194.219 48.5862,194.219 
 48.7718,194.219 48.9574,194.219 49.143,194.219 49.3286,194.219 49.5142,194.219 49.6998,194.219 49.8853,194.219 50.0709,194.219 50.2565,194.219 50.4421,194.219 
 50.6277,194.219 50.8133,194.219 50.9989,194.219 51.1844,194.219 51.37,194.219 51.5556,194.219 51.7412,194.219 51.9268,194.219 52.1124,194.219 52.298,194.219 
 52.4835,194.219 52.6691,194.219 52.8547,194.219 53.0403,194.219 53.2259,194.219 53.4115,194.219 53.597,194.219 53.7826,194.219 53.9682,194.219 54.1538,194.219 
 54.3394,194.219 54.525,194.219 54.7106,194.219 54.8961,194.219 55.0817,194.219 55.2673,194.219 55.4529,194.219 55.6385,194.219 55.8241,194.219 56.0097,194.219 
 56.1952,194.219 56.3808,194.219 56.5664,194.219 56.752,194.219 56.9376,194.219 57.1232,194.219 57.3088,194.219 57.4943,194.219 57.6799,194.219 57.8655,194.219 
 58.0511,194.219 58.2367,194.219 58.4223,194.219 58.6079,194.219 58.7934,194.219 58.979,194.219 59.1646,194.219 59.3502,194.219 59.5358,194.219 59.7214,194.219 
 59.907,194.219 60.0925,194.219 60.2781,194.219 60.4637,194.219 60.6493,194.219 60.8349,194.219 61.0205,194.219 61.206,194.219 61.3916,194.219 61.5772,194.219 
 61.7628,194.219 61.9484,194.219 62.134,194.219 62.3196,194.219 62.5051,194.219 62.6907,194.219 62.8763,194.219 63.0619,194.219 63.2475,194.219 63.4331,194.219 
 63.6187,194.219 63.8042,194.219 63.9898,194.219 64.1754,194.219 64.361,194.219 64.5466,194.219 64.7322,194.219 64.9178,194.219 65.1033,194.219 65.2889,194.219 
 65.4745,194.219 65.6601,194.219 65.8457,194.219 66.0313,194.219 66.2169,194.219 66.4024,194.219 66.588,194.219 66.7736,194.219 66.9592,194.219 67.1448,194.219 
 67.3304,194.219 67.5159,194.219 67.7015,194.219 67.8871,194.219 68.0727,194.219 68.2583,194.219 68.4439,194.219 68.6295,194.219 68.815,194.219 69.0006,194.219 
 69.1862,194.219 69.3718,194.219 69.5574,194.219 69.743,194.219 69.9286,194.219 70.1141,194.219 70.2997,194.219 70.4853,194.219 70.6709,194.219 70.8565,194.219 
 71.0421,194.219 71.2277,194.219 71.4132,194.219 71.5988,194.219 71.7844,194.219 71.97,194.219 72.1556,194.219 72.3412,194.219 72.5268,194.219 72.7123,194.219 
 72.8979,194.219 73.0835,194.219 73.2691,194.219 73.4547,194.219 73.6403,194.219 73.8259,194.219 74.0114,194.219 74.197,194.219 74.3826,194.219 74.5682,194.219 
 74.7538,194.219 74.9394,194.219 75.1249,194.219 75.3105,194.219 75.4961,194.219 75.6817,194.219 75.8673,194.219 76.0529,194.219 76.2385,194.219 76.424,194.219 
 76.6096,194.219 76.7952,194.219 76.9808,194.219 77.1664,194.219 77.352,194.219 77.5376,194.219 77.7231,194.219 77.9087,194.219 78.0943,194.219 78.2799,194.219 
 78.4655,194.219 78.6511,194.219 78.8367,194.219 79.0222,194.219 79.2078,194.219 79.3934,194.219 79.579,194.219 79.7646,194.219 79.9502,194.219 80.1358,194.219 
 80.3213,194.219 80.5069,194.219 80.6925,194.219 80.8781,194.219 81.0637,194.219 81.2493,194.219 81.4348,194.219 81.6204,194.219 81.806,194.219 81.9916,194.219 
 82.1772,194.219 82.3628,194.219 8

In [56]:
Ca=(1-((sigma*dt)./(2*eps0*epsilon)))./(1+((sigma*dt)./(2*eps0*epsilon)))
Cb = (dt./(eps0*dz*epsilon))./(1+((sigma*dt)./(2*eps0*epsilon)))
Da = (1-sigma*dt./2mu)./(1+sigma*dt./(2*mu0*mu))
Db = (dt./(mu*mu0*dz))./(1+sigma*dt./(2*mu0*mu))

# Ex[1,Int(round(length(x)/4))]+=signal_gen(t[1],frequency)
# Ex[1,Int(round(length(x)/4))+1]+=signal_gen(t[1],frequency)
Ex[1,Int(round(length(x)/4))]+=signal[1]
Ex[1,Int(round(length(x)/4))+1]+=signal[1]

for n = 2:timeSteps #Расчет
    Ex[n,1] = Ex[n-1,2] + ((c*dt-dz)/(c*dt+dz))*(Ex[n,2]-Ex[n-1,1]) #Левая граница в настоящий момент времени. Граничные условия ABC
    Ex[n,2:xSteps] = Ca[2:xSteps].*Ex[n-1,2:xSteps] - Cb[2:xSteps].*(Hy[n-1,2:xSteps]-Hy[n-1,1:xSteps-1]) #Расчет Ex в данный момент времени по всему пространству. Зависит от Hy в прошлый половинный момент времени
    
#     Ex[n,Int(round(length(x)/4))]+=signal_gen(t[n],frequency)
#     Ex[n,Int(round(length(x)/4))+1]+=signal_gen(t[n],frequency)
    Ex[n,signal_pos]+=signal[n]
    Ex[n,signal_pos+1]+=signal[n]
    
    Hy[n,1:xSteps-1] = Da[1:xSteps-1].*Hy[n-1,1:xSteps-1] - Db[1:xSteps-1].*(Ex[n,2:xSteps]-Ex[n,1:xSteps-1]) #Расчет Hy в данный момент времени по всему пространству
    Hy[n,xSteps] = Hy[n-1,xSteps-1] + ((c*dt-dz)/(c*dt+dz))*(Hy[n,xSteps-1]-Hy[n-1,xSteps]) #Правая граница в настоящий момент времени. Граничные условия -- ABC
end

# toc()


In [57]:
save("data.jld", "Ex", Ex, "Hy", Hy, "dt", dt, "dz", dz, "G", G, "x", collect(x), "t", collect(t),"signal",signal,"signal_pos",signal_pos,"xSteps", xSteps, "timeSteps", timeSteps,"left_bord_ind",left_bord_ind,"right_bord_ind",right_bord_ind)

In [58]:
plot(t*10^6,Ex[:,Int(round(length(x)/2))])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,194.444 39.6781,194.444 39.8637,194.444 40.0493,194.444 40.2349,194.444 40.4205,194.444 40.6061,194.444 40.7917,194.444 40.9772,194.444 41.1628,194.444 
 41.3484,194.444 41.534,194.444 41.7196,194.444 41.9052,194.444 42.0908,194.444 42.2763,194.444 42.4619,194.444 42.6475,194.444 42.8331,194.444 43.0187,194.444 
 43.2043,194.444 43.3899,194.444 43.5754,194.444 43.761,194.444 43.9466,194.444 44.1322,194.444 44.3178,194.444 44.5034,194.444 44.689,194.444 44.8745,194.444 
 45.0601,194.444 45.2457,194.444 45.4313,194.444 45.6169,194.444 45.8025,194.444 45.9881,194.444 46.1736,194.444 46.3592,194.444 46.5448,194.444 46.7304,194.444 
 46.916,194.444 47.1016,194.444 47.2871,194.444 47.4727,194.444 47.6583,194.444 47.8439,194.444 48.0295,194.444 48.2151,194.444 48.4007,194.444 48.5862,194.444 
 48.7718,194.444 48.9574,194.444 49.143,194.444 49.3286,194.444 49.5142,194.444 49.6998,194.444 49.8853,194.444 50.0709,194.444 50.2565,194.444 50.4421,194.444 
 50.6277,194.444 50.8133,194.444 50.9989,194.444 51.1844,194.444 51.37,194.444 51.5556,194.444 51.7412,194.444 51.9268,194.444 52.1124,194.444 52.298,194.444 
 52.4835,194.444 52.6691,194.444 52.8547,194.444 53.0403,194.444 53.2259,194.444 53.4115,194.444 53.597,194.444 53.7826,194.444 53.9682,194.444 54.1538,194.444 
 54.3394,194.444 54.525,194.444 54.7106,194.444 54.8961,194.444 55.0817,194.444 55.2673,194.444 55.4529,194.444 55.6385,194.444 55.8241,194.444 56.0097,194.444 
 56.1952,194.444 56.3808,194.444 56.5664,194.444 56.752,194.444 56.9376,194.444 57.1232,194.444 57.3088,194.444 57.4943,194.444 57.6799,194.444 57.8655,194.444 
 58.0511,194.444 58.2367,194.444 58.4223,194.444 58.6079,194.444 58.7934,194.444 58.979,194.444 59.1646,194.444 59.3502,194.444 59.5358,194.444 59.7214,194.444 
 59.907,194.444 60.0925,194.444 60.2781,194.444 60.4637,194.444 60.6493,194.444 60.8349,194.444 61.0205,194.444 61.206,194.444 61.3916,194.444 61.5772,194.444 
 61.7628,194.444 61.9484,194.444 62.134,194.444 62.3196,194.444 62.5051,194.444 62.6907,194.444 62.8763,194.444 63.0619,194.444 63.2475,194.444 63.4331,194.444 
 63.6187,194.444 63.8042,194.444 63.9898,194.444 64.1754,194.444 64.361,194.444 64.5466,194.444 64.7322,194.444 64.9178,194.444 65.1033,194.444 65.2889,194.444 
 65.4745,194.444 65.6601,194.444 65.8457,194.444 66.0313,194.444 66.2169,194.444 66.4024,194.444 66.588,194.444 66.7736,194.444 66.9592,194.444 67.1448,194.444 
 67.3304,194.444 67.5159,194.444 67.7015,194.444 67.8871,194.444 68.0727,194.444 68.2583,194.444 68.4439,194.444 68.6295,194.444 68.815,194.444 69.0006,194.444 
 69.1862,194.444 69.3718,194.444 69.5574,194.444 69.743,194.444 69.9286,194.444 70.1141,194.444 70.2997,194.444 70.4853,194.444 70.6709,194.444 70.8565,194.444 
 71.0421,194.444 71.2277,194.444 71.4132,194.444 71.5988,194.444 71.7844,194.444 71.97,194.444 72.1556,194.444 72.3412,194.444 72.5268,194.444 72.7123,194.444 
 72.8979,194.444 73.0835,194.444 73.2691,194.444 73.4547,194.444 73.6403,194.444 73.8259,194.444 74.0114,194.444 74.197,194.444 74.3826,194.444 74.5682,194.444 
 74.7538,194.444 74.9394,194.444 75.1249,194.444 75.3105,194.444 75.4961,194.444 75.6817,194.444 75.8673,194.444 76.0529,194.444 76.2385,194.444 76.424,194.444 
 76.6096,194.444 76.7952,194.444 76.9808,194.444 77.1664,194.444 77.352,194.444 77.5376,194.444 77.7231,194.444 77.9087,194.444 78.0943,194.444 78.2799,194.444 
 78.4655,194.444 78.6511,194.444 78.8367,194.444 79.0222,194.444 79.2078,194.444 79.3934,194.444 79.579,194.444 79.7646,194.444 79.9502,194.444 80.1358,194.444 
 80.3213,194.444 80.5069,194.444 80.6925,194.444 80.8781,194.444 81.0637,194.444 81.2493,194.444 81.4348,194.444 81.6204,194.444 81.806,194.444 81.9916,194.444 
 82.1772,194.444 82.3628,194.444 8

In [61]:
sr=round(1/dt)

1.19999639999e11

In [68]:
plot(Ex[750:1500,Int(round(length(x)/2))])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,194.219 40.2347,194.21 40.9768,194.201 41.7188,194.191 42.4609,194.181 43.203,194.171 43.9451,194.161 44.6872,194.151 45.4293,194.141 46.1714,194.133 
 46.9135,194.125 47.6556,194.118 48.3977,194.112 49.1398,194.108 49.8819,194.105 50.624,194.104 51.3661,194.106 52.1082,194.109 52.8503,194.115 53.5923,194.123 
 54.3344,194.133 55.0765,194.146 55.8186,194.161 56.5607,194.179 57.3028,194.198 58.0449,194.219 58.787,194.242 59.5291,194.267 60.2712,194.292 61.0133,194.318 
 61.7554,194.344 62.4975,194.37 63.2396,194.395 63.9817,194.42 64.7238,194.442 65.4659,194.462 66.2079,194.479 66.95,194.493 67.6921,194.503 68.4342,194.509 
 69.1763,194.51 69.9184,194.506 70.6605,194.496 71.4026,194.481 72.1447,194.461 72.8868,194.434 73.6289,194.402 74.371,194.363 75.1131,194.32 75.8552,194.272 
 76.5973,194.219 77.3394,194.163 78.0814,194.103 78.8235,194.041 79.5656,193.978 80.3077,193.915 81.0498,193.852 81.7919,193.792 82.534,193.735 83.2761,193.682 
 84.0182,193.635 84.7603,193.595 85.5024,193.563 86.2445,193.541 86.9866,193.529 87.7287,193.528 88.4708,193.539 89.2129,193.563 89.9549,193.6 90.697,193.651 
 91.4391,193.715 92.1812,193.792 92.9233,193.882 93.6654,193.984 94.4075,194.097 95.1496,194.219 95.8917,194.35 96.6338,194.487 97.3759,194.629 98.118,194.773 
 98.8601,194.917 99.6022,195.058 100.344,195.193 101.086,195.321 101.828,195.438 102.571,195.541 103.313,195.628 104.055,195.696 104.797,195.742 105.539,195.766 
 106.281,195.764 107.023,195.734 107.765,195.677 108.507,195.591 109.249,195.476 109.991,195.332 110.734,195.159 111.476,194.959 112.218,194.734 112.96,194.487 
 113.702,194.219 114.444,193.935 115.186,193.638 115.928,193.333 116.67,193.025 117.412,192.719 118.155,192.42 118.897,192.133 119.639,191.866 120.381,191.623 
 121.123,191.411 121.865,191.233 122.607,191.097 123.349,191.007 124.091,190.966 124.833,190.978 125.575,191.047 126.318,191.175 127.06,191.362 127.802,191.608 
 128.544,191.914 129.286,192.277 130.028,192.693 130.77,193.16 131.512,193.671 132.254,194.22 132.996,194.8 133.738,195.403 134.481,196.02 135.223,196.64 
 135.965,197.253 136.707,197.848 137.449,198.414 138.191,198.94 138.933,199.414 139.675,199.826 140.417,200.165 141.159,200.42 141.902,200.585 142.644,200.649 
 143.386,200.608 144.128,200.457 144.87,200.191 145.612,199.811 146.354,199.315 147.096,198.707 147.838,197.992 148.58,197.175 149.322,196.267 150.065,195.276 
 150.807,194.218 151.549,193.105 152.291,191.954 153.033,190.783 153.775,189.612 154.517,188.459 155.259,187.346 156.001,186.293 156.743,185.322 157.485,184.453 
 158.228,183.705 158.97,183.098 159.712,182.648 160.454,182.372 161.196,182.281 161.938,182.387 162.68,182.696 163.422,183.214 164.164,183.941 164.906,184.876 
 165.649,186.011 166.391,187.336 167.133,188.84 167.875,190.503 168.617,192.305 169.359,194.222 170.101,196.227 170.843,198.29 171.585,200.379 172.327,202.459 
 173.069,204.494 173.812,206.45 174.554,208.287 175.296,209.972 176.038,211.468 176.78,212.742 177.522,213.762 178.264,214.501 179.006,214.934 179.748,215.041 
 180.49,214.805 181.232,214.216 181.975,213.27 182.717,211.967 183.459,210.313 184.201,208.323 184.943,206.015 185.685,203.416 186.427,200.557 187.169,197.475 
 187.911,194.214 188.653,190.82 189.396,187.345 190.138,183.845 190.88,180.377 191.622,177 192.364,173.775 193.106,170.762 193.848,168.019 194.59,165.602 
 195.332,163.566 196.074,161.958 196.816,160.821 197.559,160.194 198.301,160.104 199.043,160.575 199.785,161.618 200.527,163.239 201.269,165.431 202.011,168.178 
 202.753,171.456 203.495,175.228 204.237,179.45 204.98,184.067 205.722,189.017 206.464,194.228 207.206,199.625 207.948,205.122 208.69,210.632 209.432,216.064 
 210.174,221.325 210.916,226.322 211.658,230.9

In [70]:
plot(Ex[1501:2251,Int(round(length(x)/2))])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,194.619 40.2347,194.575 40.9768,194.536 41.7188,194.502 42.4609,194.473 43.203,194.449 43.9451,194.431 44.6872,194.419 45.4293,194.411 46.1714,194.409 
 46.9135,194.411 47.6556,194.418 48.3977,194.428 49.1398,194.442 49.8819,194.458 50.624,194.478 51.3661,194.498 52.1082,194.521 52.8503,194.544 53.5923,194.567 
 54.3344,194.591 55.0765,194.614 55.8186,194.636 56.5607,194.656 57.3028,194.675 58.0449,194.692 58.787,194.707 59.5291,194.72 60.2712,194.731 61.0133,194.739 
 61.7554,194.745 62.4975,194.749 63.2396,194.751 63.9817,194.75 64.7238,194.748 65.4659,194.744 66.2079,194.738 66.95,194.732 67.6921,194.724 68.4342,194.716 
 69.1763,194.708 69.9184,194.699 70.6605,194.69 71.4026,194.682 72.1447,194.675 72.8868,194.669 73.6289,194.663 74.371,194.659 75.1131,194.656 75.8552,194.654 
 76.5973,194.654 77.3394,194.655 78.0814,194.658 78.8235,194.662 79.5656,194.668 80.3077,194.674 81.0498,194.682 81.7919,194.69 82.534,194.699 83.2761,194.708 
 84.0182,194.717 84.7603,194.726 85.5024,194.735 86.2445,194.743 86.9866,194.749 87.7287,194.755 88.4708,194.758 89.2129,194.76 89.9549,194.76 90.697,194.758 
 91.4391,194.753 92.1812,194.746 92.9233,194.737 93.6654,194.725 94.4075,194.71 95.1496,194.694 95.8917,194.675 96.6338,194.654 97.3759,194.631 98.118,194.607 
 98.8601,194.583 99.6022,194.557 100.344,194.532 101.086,194.507 101.828,194.483 102.571,194.461 103.313,194.44 104.055,194.422 104.797,194.408 105.539,194.397 
 106.281,194.39 107.023,194.388 107.765,194.391 108.507,194.399 109.249,194.413 109.991,194.433 110.734,194.458 111.476,194.489 112.218,194.526 112.96,194.569 
 113.702,194.616 114.444,194.668 115.186,194.724 115.928,194.783 116.67,194.844 117.412,194.907 118.155,194.97 118.897,195.032 119.639,195.093 120.381,195.15 
 121.123,195.203 121.865,195.25 122.607,195.29 123.349,195.322 124.091,195.345 124.833,195.357 125.575,195.358 126.318,195.347 127.06,195.324 127.802,195.287 
 128.544,195.237 129.286,195.173 130.028,195.096 130.77,195.006 131.512,194.905 132.254,194.792 132.996,194.67 133.738,194.539 134.481,194.402 135.223,194.261 
 135.965,194.117 136.707,193.973 137.449,193.832 138.191,193.697 138.933,193.569 139.675,193.452 140.417,193.349 141.159,193.262 141.902,193.194 142.644,193.147 
 143.386,193.124 144.128,193.126 144.87,193.155 145.612,193.212 146.354,193.298 147.096,193.413 147.838,193.558 148.58,193.73 149.322,193.93 150.065,194.155 
 150.807,194.402 151.549,194.67 152.291,194.954 153.033,195.251 153.775,195.556 154.517,195.864 155.259,196.17 156.001,196.469 156.743,196.755 157.485,197.022 
 158.228,197.265 158.97,197.478 159.712,197.655 160.454,197.791 161.196,197.882 161.938,197.923 162.68,197.91 163.422,197.841 164.164,197.714 164.906,197.527 
 165.649,197.28 166.391,196.975 167.133,196.612 167.875,196.196 168.617,195.729 169.359,195.218 170.101,194.669 170.843,194.089 171.585,193.486 172.327,192.869 
 173.069,192.249 173.812,191.636 174.554,191.041 175.296,190.475 176.038,189.949 176.78,189.475 177.522,189.063 178.264,188.724 179.006,188.469 179.748,188.304 
 180.49,188.24 181.232,188.281 181.975,188.432 182.717,188.698 183.459,189.078 184.201,189.574 184.943,190.182 185.685,190.897 186.427,191.714 187.169,192.622 
 187.911,193.612 188.653,194.671 189.396,195.784 190.138,196.935 190.88,198.105 191.622,199.277 192.364,200.43 193.106,201.543 193.848,202.596 194.59,203.567 
 195.332,204.436 196.074,205.184 196.816,205.791 197.559,206.24 198.301,206.517 199.043,206.608 199.785,206.502 200.527,206.193 201.269,205.675 202.011,204.947 
 202.753,204.013 203.495,202.878 204.237,201.552 204.98,200.049 205.722,198.386 206.464,196.584 207.206,194.667 207.948,192.661 208.69,190.599 209.432,188.51 
 210.174,186.43 210.916,184.394 211.658,182

In [83]:
S1=abs(fft(Ex[750:1499,Int(round(length(x)/2))]));
S2=abs(fft(Ex[1500:2249,Int(round(length(x)/2))]));

In [84]:
df=sr/750

1.5999951999866667e8

In [82]:
plot([S1[1:325],S2[1:325]])
# plot(S2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 50 
 
 
 100 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,384.913 32.3489,384.913 34.0942,384.912 35.8394,384.912 37.5847,384.91 39.3299,384.909 41.0751,384.906 42.8204,384.903 44.5656,384.899 46.3109,384.895 
 48.0561,384.832 49.8014,383.691 51.5466,369.211 53.2919,293.655 55.0371,120.03 56.7823,3.93701 58.5276,112.496 60.2728,288.373 62.0181,367.845 63.7633,383.52 
 65.5086,384.837 67.2538,384.907 68.9991,384.913 70.7443,384.918 72.4895,384.922 74.2348,384.926 75.98,384.929 77.7253,384.931 79.4705,384.934 81.2158,384.935 
 82.961,384.937 84.7063,384.938 86.4515,384.939 88.1968,384.94 89.942,384.941 91.6872,384.942 93.4325,384.943 95.1777,384.943 96.923,384.944 98.6682,384.944 
 100.413,384.945 102.159,384.945 103.904,384.945 105.649,384.946 107.394,384.946 109.14,384.946 110.885,384.947 112.63,384.947 114.375,384.947 116.121,384.947 
 117.866,384.948 119.611,384.948 121.356,384.948 123.102,384.948 124.847,384.948 126.592,384.948 128.337,384.949 130.083,384.949 131.828,384.949 133.573,384.949 
 135.318,384.949 137.064,384.949 138.809,384.949 140.554,384.949 142.299,384.949 144.045,384.95 145.79,384.95 147.535,384.95 149.28,384.95 151.026,384.95 
 152.771,384.95 154.516,384.95 156.261,384.95 158.007,384.95 159.752,384.95 161.497,384.95 163.242,384.95 164.988,384.95 166.733,384.95 168.478,384.95 
 170.223,384.95 171.969,384.95 173.714,384.95 175.459,384.951 177.204,384.951 178.949,384.951 180.695,384.951 182.44,384.951 184.185,384.951 185.93,384.951 
 187.676,384.951 189.421,384.951 191.166,384.951 192.911,384.951 194.657,384.951 196.402,384.951 198.147,384.951 199.892,384.951 201.638,384.951 203.383,384.951 
 205.128,384.951 206.873,384.951 208.619,384.951 210.364,384.951 212.109,384.951 213.854,384.951 215.6,384.951 217.345,384.951 219.09,384.951 220.835,384.951 
 222.581,384.951 224.326,384.951 226.071,384.951 227.816,384.951 229.562,384.951 231.307,384.951 233.052,384.951 234.797,384.951 236.543,384.951 238.288,384.951 
 240.033,384.951 241.778,384.951 243.524,384.951 245.269,384.951 247.014,384.951 248.759,384.951 250.505,384.951 252.25,384.951 253.995,384.951 255.74,384.951 
 257.485,384.951 259.231,384.951 260.976,384.951 262.721,384.951 264.466,384.951 266.212,384.951 267.957,384.951 269.702,384.951 271.447,384.951 273.193,384.951 
 274.938,384.951 276.683,384.951 278.428,384.951 280.174,384.952 281.919,384.952 283.664,384.952 285.409,384.952 287.155,384.952 288.9,384.952 290.645,384.952 
 292.39,384.952 294.136,384.952 295.881,384.952 297.626,384.952 299.371,384.952 301.117,384.952 302.862,384.952 304.607,384.952 306.352,384.952 308.098,384.952 
 309.843,384.952 311.588,384.952 313.333,384.952 315.079,384.952 316.824,384.952 318.569,384.952 320.314,384.952 322.06,384.952 323.805,384.952 325.55,384.952 
 327.295,384.952 329.041,384.952 330.786,384.952 332.531,384.952 334.276,384.952 336.022,384.952 337.767,384.952 339.512,384.952 341.257,384.952 343.002,384.952 
 344.748,384.952 346.493,384.952 348.238,384.952 349.983,384.952 351.729,384.952 353.474,384.952 355.219,384.952 356.964,384.952 358.71,384.952 360.455,384.952 
 362.2,384.952 363.945,384.952 365.691,384.952 367.436,384.952 369.181,384.952 370.926,384.952 372.672,384.952 374.417,384.952 376.162,384.952 377.907,384.952 
 379.653,384.952 381.398,384.952 383.143,384.952 384.888,384.952 386.634,384.952 388.379,384.952 390.124,384.952 391.869,384.952 393.615,384.952 395.36,384.952 
 397.105,384.952 398.85,384.952 400.596,384.952 402.341,384.952 404.086,384.952 405.831,384.952 407.577,384.952 409.322,384.952 411.067,384.952 412.812,384.952 
 414.558,384.952 416.303,384.952 418.048,384.952 419.793,384.952 421.539,384.952 423.284,384.952 425.029,384.952 426.774,384.952 428.519,384.952 430.265,384.952 
 432.01,384.952 433.755,384.952 435.5,384.952 437.246,384.95